In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
from mt import Evolver, Teacher, SpacyTokenizer

tokenizer = SpacyTokenizer()
config = {
    'd_model': 64, 'dim_feedforward': 256, 'encoder_layers': 2, 'decoder_layers': 2, 'nhead': 1, 'dropout': 0.1, 'layer_norm_eps': 1e-5, 'max_len': 32,
    'pad_token_id': tokenizer.pad_token_id, 'eos_token_id': tokenizer.eos_token_id, 'bos_token_id': tokenizer.bos_token_id, 'vocab_size': tokenizer.vocab_size, 'name' : None}

student = Evolver(**config)
teacher = Teacher(**config)

In [ ]:
from mt import TrajectoryDataset
from torch.utils.data import DataLoader

dataset = TrajectoryDataset(split='test', truncate=4, tokenizer=tokenizer, max_len=32)
loader = DataLoader(dataset, batch_size=2, collate_fn=dataset.collate_fn)

In [ ]:
from tqdm import tqdm
from itertools import islice

visited = []
for batch in tqdm(islice(loader, 5), desc='rolling out student'):
    for t in student.rollout(batch, T=3)[1:-1]:
        visited.extend({'src_ids': batch['src_ids'][i], 'input_ids': t[i]} for i in range(2))

In [32]:
visited_dataset = DataLoader(visited, batch_size=2, collate_fn=dataset.collate_fn_reduced)

In [ ]:
new_dataset = []
for batch in tqdm(visited_dataset, desc='generating teacher labels'):
    tgt_ids = teacher._generate(batch)
    new_dataset.extend(
        {'src_ids': batch['src_ids'][i], 'input_ids': batch['input_ids'][i], 'tgt_ids': tgt_ids[i]}
        for i in range(2))